In [ ]:
import os
import re
import pandas as pd
import unicodedata
import psycopg2
from psycopg2.extras import execute_values
from pathlib import Path
from tqdm import tqdm
from IPython.display import display, Markdown
from datetime import datetime


def agora():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


# --------- 1. FUNÇÕES UTILITÁRIAS ---------

def limpar_coluna(col):
    col = col.strip().lower()
    col = unicodedata.normalize('NFKD', col).encode('ascii', 'ignore').decode('utf-8')

    col = col.replace("r$", "rs").replace("u$", "us")
    col = col.replace("(", "").replace(")", "").replace("*", "")
    col = re.sub(r'[^a-z0-9]+', '_', col)     # substitui qualquer sequência de não-letras/números por "_"
    col = re.sub(r'_+', '_', col)             # substitui múltiplos "_" por um só
    col = col.strip('_')                      # remove "_" do início e fim, se houver

    # ajuste específico para 'programa_desligamento_voluntario'
    col = col.replace('programa_desligamento_voluntario', 'pdv')
    col = col.replace('verbas_indenizatorias_registradas_em_sistemas_de_pessoal', 'verbas_indenizatorias')
    

    return col

def para_float(valor):
    try:
        if isinstance(valor, str):
            valor = valor.replace('.', '').replace(',', '.')
        return float(valor)
    except:
        return None

def gerar_create_table_sql(df, nome_tabela="siape"):
    tipo_sql = {
        "object": "TEXT",
        "int64": "NUMERIC",
        "float64": "NUMERIC",
        "datetime64[ns]": "DATE",
        "bool": "BOOLEAN"
    }

    colunas_sql = []
    for coluna, tipo in df.dtypes.items():
        tipo_postgres = tipo_sql.get(str(tipo), "TEXT")
        colunas_sql.append(f'    {coluna} {tipo_postgres}')

    colunas_str = ",\n".join(colunas_sql)
    
    sql = f"""
        CREATE TABLE IF NOT EXISTS {nome_tabela} (
            id SERIAL PRIMARY KEY,
            {colunas_str}
        ) TABLESPACE siape_space;
    """
    
    return sql

def dados_ja_inseridos(ano, mes):
    try:
        display(Markdown(f"{agora()} - dados_ja_inseridos - conectando"))
        con = psycopg2.connect(dbname="siape", user="bonafe", password="ovopreto", host="localhost", port="5432")
        cur = con.cursor()
        display(Markdown(f"{agora()} - dados_ja_inseridos - select"))
        cur.execute("SELECT COUNT(*) FROM siape WHERE ano = %s AND mes = %s", (str(ano), mes))
        count = cur.fetchone()[0]
        cur.close()
        con.close()
        return count > 0
    except:
        return False

def inserir_siape_no_banco(df, ano, mes, chunk_size=5000):
    df.columns = [limpar_coluna(c) for c in df.columns]
    df["ano"] = str(ano)
    df["mes"] = int(mes)

    display(Markdown(f"{agora()} - transformando em float"))
    for col in [c for c in df.columns if "rs" in c or "us" in c or "remuneracao" in c or "verbas" in c]:
        df[col] = df[col].apply(para_float)

    display(Markdown(f"{agora()} - transformando em data"))
    for col in [c for c in df.columns if "data_" in c]:
        df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)
    if "id" in df.columns:
        df.drop(columns=["id"], inplace=True)

    colunas = list(df.columns)
    total = len(df)

    try:
        con = psycopg2.connect(dbname="siape", user="bonafe", password="ovopreto", host="localhost", port="5432")
        cur = con.cursor()
        for i in tqdm(range(0, total, chunk_size), desc=f"Inserindo {mes}/{ano}"):
            chunk = df.iloc[i:i+chunk_size]            
            valores = chunk.replace({pd.NaT: None}).where(pd.notnull(chunk), None).values.tolist()
            insert_query = f"INSERT INTO siape ({', '.join(colunas)}) VALUES %s"            
            execute_values(cur, insert_query, valores)
        con.commit()
        display(Markdown(f"✅ **Inseridos {total} registros do mês {mes}/{ano}.**"))
    except Exception as e:
        display(Markdown(f"❌ **Erro ao inserir {mes}/{ano}:** {e}"))
    finally:
        cur.close()
        con.close()


# --------- 2. CRIA TABELA BASEADO EM UM MÊS DE EXEMPLO ---------

base_dir = Path("../../../dados/dados_siape_bruto/descompactado")
pastas = sorted(base_dir.glob("*_Servidores_SIAPE"))

if not pastas:
    raise Exception("Nenhuma pasta *_Servidores_SIAPE encontrada.")


# Cria ou recria a tabela no banco
try:
    con = psycopg2.connect(dbname="siape", user="bonafe", password="ovopreto", host="localhost", port="5432")
    cur = con.cursor()
    cur.execute("SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_name = 'siape');")
    existe = cur.fetchone()[0]

    if existe:
        
        resposta = input("A tabela 'siape' já existe. Deseja apagá-la e recriar? (s/n): ").strip().lower()
        
        if resposta == 's':
            
            cur.execute("DROP TABLE IF EXISTS siape CASCADE;")
            pasta_exemplo = pastas[0]
            ano_mes = pasta_exemplo.name.split("_")[0]
            ano = int(ano_mes[:4])
            mes = int(ano_mes[4:6])
            cadastro_csv = pasta_exemplo / f"{ano_mes}_Cadastro.csv"
            remuneracao_csv = pasta_exemplo / f"{ano_mes}_Remuneracao.csv"
            
            df_c = pd.read_csv(cadastro_csv, encoding="iso-8859-1", sep=";", low_memory=False)
            df_r = pd.read_csv(remuneracao_csv, encoding="iso-8859-1", sep=";", low_memory=False)
            df = pd.merge(df_c, df_r, on="Id_SERVIDOR_PORTAL", how="left")
            
            # 🧼 Corrige colunas ANTES de gerar SQL
            df.columns = [limpar_coluna(c) for c in df.columns]
                
            
            df["ano"] = int(ano)
            df["mes"] = int(mes)
            
            
            for col in [c for c in df.columns if "rs" in c or "us" in c or "remuneracao" in c or "verbas" in c]:
                df[col] = df[col].apply(para_float)
            for col in [c for c in df.columns if "data_" in c]:
                df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)    
            
            # ✅ Agora sim, gerar o SQL com colunas já limpas
            create_table_sql = gerar_create_table_sql(df)
                
            cur.execute(create_table_sql)    
            con.commit()
            print("🔁 Tabela antiga removida.")
            
        else:
            print("✅ Mantida a tabela existente. Nenhuma alteração.")
    
    cur.close()
    con.close()
    display(Markdown("✅ **Tabela `siape` verificada/criada com sucesso.**"))
except Exception as e:
    display(Markdown(f"❌ **Erro ao criar a tabela:** {e}"))


# --------- 3. INSERE OS DADOS DE TODOS OS MESES ---------

for pasta in pastas:
    try:
        ano_mes = pasta.name.split("_")[0]
        ano = int(ano_mes[:4])
        mes = int(ano_mes[4:6])

                
        display(Markdown(f"{agora()} - iniciando"))
        
        if dados_ja_inseridos(ano, mes):
            display(Markdown("ℹ️ **Dados já inseridos anteriormente. Pulando.**"))
            continue

        cadastro_csv = pasta / f"{ano_mes}_Cadastro.csv"
        remuneracao_csv = pasta / f"{ano_mes}_Remuneracao.csv"

        if not cadastro_csv.exists() or not remuneracao_csv.exists():
            display(Markdown("⚠️ **Arquivos não encontrados. Pulando.**"))
            continue

        display(Markdown(f"{agora()} - lendo cadastro"))
        df_c = pd.read_csv(cadastro_csv, encoding="iso-8859-1", sep=";", low_memory=False)
        display(Markdown(f"{agora()} - lendo remuneração"))
        df_r = pd.read_csv(remuneracao_csv, encoding="iso-8859-1", sep=";", low_memory=False)
        display(Markdown(f"{agora()} - fazendo merge"))
        df = pd.merge(df_c, df_r, on="Id_SERVIDOR_PORTAL", how="left")

        display(Markdown(f"{agora()} - inserir no banco"))
        inserir_siape_no_banco(df, ano, mes)

    except Exception as e:
        display(Markdown(f"❌ **Erro ao processar `{pasta.name}`:** {e}"))


A tabela 'siape' já existe. Deseja apagá-la e recriar? (s/n):  n


✅ Mantida a tabela existente. Nenhuma alteração.


✅ **Tabela `siape` verificada/criada com sucesso.**

2025-03-30 11:28:37 - iniciando

2025-03-30 11:28:37 - dados_ja_inseridos - conectando

...

2025-03-30 11:57:53 - dados_ja_inseridos - select

ℹ️ **Dados já inseridos anteriormente. Pulando.**

2025-03-30 12:11:27 - iniciando

2025-03-30 12:11:27 - dados_ja_inseridos - conectando

2025-03-30 12:11:27 - dados_ja_inseridos - select